In [1]:
import os

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

gpu_on = True

if gpu_on :
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from gan_module.data_loader.classification import ClassifyDataloader
from glob import glob

data_set_name = "detect_lvi"
batch_size = 2

train_image_path = glob(f"./datasets/{data_set_name}/train/*/*")
valid_image_path = glob(f"./datasets/{data_set_name}/valid/*/*")
label_list = os.listdir(f"./datasets/{data_set_name}/train")
label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}
train_data_loader = ClassifyDataloader(image_path_list=train_image_path,
                                       label_to_index_dict = label_to_index_dict,
                                       batch_size = batch_size,
                                       shuffle=True
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path,
                                       label_to_index_dict = label_to_index_dict,
                                       batch_size = batch_size,
                                       shuffle=True
)

In [3]:
from tensorflow.keras.applications import InceptionV3, ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.activations import sigmoid
import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

transfer_learning = True
train_mode = "deep_layer_only"

# create the base pre-trained model
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=train_data_loader.source_data_shape,
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if train_mode == "dense_only":
        base_model.trainable = False
    elif train_mode == "deep_layer_only":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == "mixed4":
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)
############## MODIFIED!!!!!!! 1 => 2 ##################
predictions = Dense(2, activation='sigmoid')(x)


# this is the model we will train
model = Model(base_model.input, predictions)

In [4]:
model.load_weights("./U_net/LVI_detect_classfication/weights_1.0_1.0_InceptionV3_Imagenet_LVI_level2/weights_111_0.0008.hdf5", by_name=True, skip_mismatch=True)

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import Nadam

loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01)
optimizer = Nadam(1e-3, clipnorm=1)

# save_c = ModelCheckpoint(
#     "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
#     monitor='val_loss',
#     verbose=0,
#     save_best_only=False,
#     save_weights_only=True,
#     mode='min')

save_c = ModelCheckpoint(
    "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=10,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-9)
csv_logger = CSVLogger('./U_net/log.csv', append=False, separator=',')

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
start_epoch = 0
epochs = 200

model.fit(
    train_data_loader,
    validation_data=valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger],
    initial_epoch=start_epoch
)

Epoch 1/200
4304/4304 [==============================] - 784s 179ms/step - loss: 0.7262 - accuracy: 0.5776 - val_loss: 10.8549 - val_accuracy: 0.6542
Epoch 2/200
4304/4304 [==============================] - 758s 176ms/step - loss: 0.6818 - accuracy: 0.6332 - val_loss: 10.9857 - val_accuracy: 0.5566
Epoch 3/200
4304/4304 [==============================] - 776s 180ms/step - loss: 0.6398 - accuracy: 0.6985 - val_loss: 2.8217 - val_accuracy: 0.7674
Epoch 4/200
4304/4304 [==============================] - 770s 179ms/step - loss: 0.4559 - accuracy: 0.8313 - val_loss: 2.6670 - val_accuracy: 0.6673
Epoch 5/200
4304/4304 [==============================] - 802s 186ms/step - loss: 0.3537 - accuracy: 0.8871 - val_loss: 0.5264 - val_accuracy: 0.7842
Epoch 6/200
4304/4304 [==============================] - 793s 184ms/step - loss: 0.2676 - accuracy: 0.9233 - val_loss: 2.0279 - val_accuracy: 0.6891
Epoch 7/200
4304/4304 [==============================] - 780s 181ms/step - loss: 0.2083 - accuracy: 0.94